
# Oh bien la montre !
Ici nous faire une demo du scraping de données sur amazon
Le but étant de trouvé des acticle classé par ordre de prix pour le moment 

In [43]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import nest_asyncio




Nous allons d'abors charger la page amazon qui contient tout les prooduits...
Nous utilison le module requests_html car il permet d'avoir le user agent configurer

In [44]:
pattern_produit="camera+vlog"
url = f"https://www.amazon.fr/s?k={pattern_produit}&i=electronics"


def getPageRH(url):
    nest_asyncio.apply()
    session = HTMLSession()
    reponse = session.get(url)
    reponse.html.render(sleep=1)
    soup = BeautifulSoup(reponse.html. html, 'html.parser')
    return soup



Nous pouvons afficher les produits avec leurs titre leur prix, le nombre d'avis et la note de la page, sans le contenu sponsorisé


In [45]:
def getMytitleProduct(url):
    page = getPageRH(url)
    for item in page.find('div', {'class' : 's-main-slot s-result-list s-search-results sg-row'}).findAll({'div'}) : 
        if item.get("data-component-type") :
            title = item.find('a', {'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'}).text.strip()[:25]
            try:
                price = item.find('span',{'class': 'a-price-whole'}).text.strip().replace(',','.')
            except AttributeError:
                price = 0
            try:
                avis = item.find('span',{'class': 'a-size-base puis-light-weight-text s-link-centralized-style'}).text.strip()
            except AttributeError:
                avis = 0
            try:
                stars = float(item.find('span',{'class':'a-icon-alt'}).text.strip()[:3].replace(',','.'))
            except AttributeError:
                stars = 0
            print(title, f"Price : {price}; Avis : {avis}; stars : {stars}\n-------")


Maintenant il faut qu'on transforme les données que nous avons recuperer pour les dans un tableau,
j'entend par la l'utilisation de pandas

In [48]:
import pandas

def getDataframeFromAmazon(url):
    list_avis = []
    list_nom = []
    list_prix = []
    list_note = []
    page = getPageRH(url)
    for item in page.find('div', {'class' : 's-main-slot s-result-list s-search-results sg-row'}).findAll({'div'}) : 
        if item.get("data-component-type") :
            title = item.find('a', {'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'}).text.strip()[:25]
            list_nom.append(title)
            try:
                price = item.find('span',{'class': 'a-price-whole'}).text.strip().replace(',','.')
            except AttributeError:
                price = 0
            list_prix.append(price)
            try:
                avis = item.find('span',{'class': 'a-size-base puis-light-weight-text s-link-centralized-style'}).text.strip()
            except AttributeError:
                avis = 0
            list_avis.append(avis)
            try:
                stars = float(item.find('span',{'class':'a-icon-alt'}).text.strip()[:3].replace(',','.'))
            except AttributeError:
                stars = 0
            list_note.append(stars)
            #print(title, f"Price : {price}; Avis : {avis}; stars : {stars}\n-------")
    df = pandas.DataFrame.from_dict( {"Product" : list_nom, "Prix" : list_prix, "Nombre_avis" : list_avis, "Note" : list_note})
    return df

La fonction permet de faire un dataframe par page, il faut donc regrouper les dataframes en un seul

In [49]:
def regroupDf(proxies,produit):
    tab = []
    for num in range(5):
        url = f"https://www.amazon.fr/s?k={produit}&i=electronics&page={str(num+1)}"
        #print(f"page {num+1}:\n")
        tab.append(getDataframeFromAmazon(url,proxies))

    df = pandas.concat(tab, sort=False)
    return df

Une fois les dataframes recuperés nous pouvons mettre dans un csv 

In [ ]:
def toCsv(df : pandas.DataFrame, path : str):
    df.to_csv(path_or_buf=path)